In [1]:
!pip install shioaji
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#登入你的key
accounts = api.login()



2024-04-07 00:54:02.868 | WARNING  | importlib._bootstrap:_call_with_frames_removed:241 - Optional: pip install shioaji[speed] for better performance.


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [3]:
from datetime import date, timedelta
today = date.today()
delta = timedelta(days=180)
date_180 = today - delta

print("今天的日期:", today)
print("180天前的日期:", date_180)


今天的日期: 2024-04-07
180天前的日期: 2023-10-10


SDK NOTICE Sun Apr 07 01:22:35.656 2024 solClientOS.c:6131                   (177737000) TCP: Could not read from socket 82, error = Connection reset by peer (54)
SDK NOTICE Sun Apr 07 01:22:35.657 2024 solClientSmf.c:7959                  (177737000) Read failed from transport 'compression' of session '(c0,s1)_sinopac' due to 'TCP: Could not read from socket 82, error = Connection reset by peer (54)', client name 'PYAPI/F129970274/0406/165403/909938/118.150.47.40', VPN name 'sinopac', peer host '203.66.91.161:80' address 'IP 203.66.91.161', connection 'tcp_TxRx' local address 'IP 192.168.100.9:50793'


Response Code: 0 | Event Code: 12 | Info: TCP: Could not read from socket 82, error = Connection reset by peer (54) | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


SDK NOTICE Sun Apr 07 01:40:09.820 2024 solClient.c:11120                    (177737000) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0406/165403/909938/118.150.47.40', VPN name 'sinopac', peer host '203.66.91.161:80' address 'IP 203.66.91.161', connection 'tcp_TxRx' local address 'IP 192.168.100.9:51721'


Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0406/165403/909938/118.150.47.40', VPN name 'sinopac', peer host '203.66.91.161:80' address 'IP 203.66.91.161', connection 'tcp_TxRx' local address 'IP 1 | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


SDK NOTICE Sun Apr 07 01:47:06.960 2024 solClient.c:11120                    (177737000) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0406/165403/909938/118.150.47.40', VPN name 'sinopac', peer host '203.66.91.161:80' address 'IP 203.66.91.161', connection 'tcp_TxRx' local address 'IP 192.168.100.9:51766'


Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0406/165403/909938/118.150.47.40', VPN name 'sinopac', peer host '203.66.91.161:80' address 'IP 203.66.91.161', connection 'tcp_TxRx' local address 'IP 1 | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


In [33]:
buy_price=int(290)
stock_code=str(2382)
def calculate_stop_loss(buy_price):
    stop_loss_five_percent = buy_price*0.95
    stop_loss_four_percent = buy_price*0.96
    return stop_loss_five_percent, stop_loss_four_percent  # 設定停損價格為買進價格的90%


def calculate_moving_averages(stock_code, window=60, buy_price=None):

    # 取得歷史股價資料
    # k棒的api使用方式
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code], 
        start=str(date_180), 
        end=str(today), 
    )
    df = pd.DataFrame({**kbars})
    df.ts = pd.to_datetime(df.ts)
    df.set_index('ts',inplace=True)

    # 日k13:30的K棒
    df = df.resample('D').last().dropna()

    

    # 計算移動平均線
    df['5MA'] = df['Close'].rolling(window=5).mean()
    df['10MA'] = df['Close'].rolling(window=10).mean()
    df['20MA'] = df['Close'].rolling(window=20).mean()
    df['60MA'] = df['Close'].rolling(window=60).mean()
    df['20MA/60MA'] = df['20MA'] / df['60MA'].rolling(window=window).mean()
    # 計算價格差距百分比
    if buy_price is not None:
        df['5MA_diff'] = (buy_price - df['5MA']) / buy_price * 100
        df['10MA_diff'] = (buy_price - df['10MA']) / buy_price * 100
        df['20MA_diff'] = (buy_price - df['20MA']) / buy_price * 100
        df['60MA_diff'] = (buy_price - df['60MA']) / buy_price * 100
        
    return df[['5MA', '10MA', '20MA', '20MA/60MA','5MA_diff','10MA_diff']]

print(calculate_stop_loss(buy_price))
print(calculate_moving_averages(stock_code, window=60, buy_price=290))

(275.5, 278.4)
              5MA    10MA     20MA  20MA/60MA   5MA_diff  10MA_diff
ts                                                                 
2023-10-03    NaN     NaN      NaN        NaN        NaN        NaN
2023-10-04    NaN     NaN      NaN        NaN        NaN        NaN
2023-10-05    NaN     NaN      NaN        NaN        NaN        NaN
2023-10-06    NaN     NaN      NaN        NaN        NaN        NaN
2023-10-11  248.2     NaN      NaN        NaN  14.413793        NaN
...           ...     ...      ...        ...        ...        ...
2024-03-25  254.6  253.40  246.925        NaN  12.206897  12.620690
2024-03-26  254.0  253.35  247.575        NaN  12.413793  12.637931
2024-03-27  255.7  254.05  248.825        NaN  11.827586  12.396552
2024-03-28  260.8  257.70  251.225        NaN  10.068966  11.137931
2024-03-29  268.0  261.30  254.050        NaN   7.586207   9.896552

[118 rows x 6 columns]
